In [7]:
import numpy as np
import pdfparser.poppler as pdf
import simple_text_representation/classes


SyntaxError: invalid syntax (<ipython-input-7-5630edb006c3>, line 3)

In [9]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

!ls

Saving textos_primero_secundaria.pdf to textos_primero_secundaria.pdf
User uploaded file "textos_primero_secundaria.pdf" with length 12006763 bytes
datalab  simple-text-representation  textos_primero_secundaria.pdf


In [0]:
INITIAL_PAGE = 5
PAGES_PER_TEXT = 2
ALLOWED_TAGS = ['APRENDEMOS', 'PRACTICAMOS', 'EXTENSIÓN']

In [0]:
def convertPDFtoText(fileName):
    document = pdf.Document(fileName.encode('UTF-8'))

    print('No of pages', document.no_of_pages)
#     for page in document:
#         print('Page', page.page_no, 'size =', page.size)
#         for f in page:
# #             print(' '*1,'Flow')
#             for b in f:
# #                 print(' '*2,'Block', 'bbox=', b.bbox.as_tuple())
#                 for l in b:
#                     print(' '*3, l.text)
#                     #assert l.char_fonts.comp_ratio < 1.0
# #                     for i in range(len(l.text)):
# #                         print(l.text[i].encode('UTF-8'), '(%0.2f, %0.2f, %0.2f, %0.2f)'% l.char_bboxes[i].as_tuple(),\
# #                             l.char_fonts[i].name, l.char_fonts[i].size, l.char_fonts[i].color,)
#                     print()
#                 print("=============================================================================")
    return document

In [12]:
document = convertPDFtoText("textos_primero_secundaria.pdf")

No of pages 194


In [0]:
def hasAllowedTag(line):
    return any(TAG in line for TAG in ALLOWED_TAGS)

In [0]:
def cleanDocument(file):
    tagFound = False
    nextPageAdded = -1
    text = list()
    listOfText = list()
    
    for index,page in enumerate(file):
        if (index > INITIAL_PAGE):
            if (nextPageAdded == 1):
                text.append(page)
                listOfText.append(text)
                text = list()
                nextPageAdded = -1
            else:                
                for flow in page:
                    for paragraph in flow:
                        for line in paragraph:
                            if (hasAllowedTag(line.text)):
                                text.append(page)
                                nextPageAdded = 1
#                             print(tagFound, line.text)
#                         print("================================================================")
    return listOfText;

In [0]:
listOfText = cleanDocument(document)

In [16]:
listOfText

firstText = listOfText[0]

print(firstText)

[<pdfparser.poppler.Page object at 0x7f0a95fabaf8>, <pdfparser.poppler.Page object at 0x7f0a95fcb798>]


In [0]:
# pdfArr = np.array(pdf)[INITIAL_PAGE:]

In [0]:
cleanTextArr = list()

for index, text in enumerate(pdfArr):
    if hasAllowedTag(text):
        text = []
        for position in range(index, index + PAGES_PER_TEXT):
            text.append(pdfArr[position])
        cleanTextArr.append(text)

len(cleanTextArr)

60

# Limpieza de Textos

## Estructura actual de un Texto

Despues de realizar la estracción de textos con la herramienta PDFtoText se consigue un listado de elementos, donde cada elemento representa una parte de un texto y cada par de elementos adyacentes representan un texto completo. En su estado original, estos traen mucha información innecesaria que debe ser removida y que detallará a continuación.

* Tema a tratar
* Etiqueta de la sección
* Indicaciones del texto
* Titulo del texto
* Referencias de imagenes
* Preguntas relacionas al texto.
* Referencias del texto
* Pie de pagina
* Descripción de la hoja actual.


## Retos de la Limpieza

* Conservar la estructura original del texto, es decir, que todos los parrafos sean identificables.
* Eliminar todas las referencias de imagenes existentes, ya que se usan de forma indeterminada en distintas partes de un texto.
* Adaptar las diferentes estructuras de los archivos PDF provistos

## Pasos de Limpieza

* Usar la Etiqueta de cada texto para determinar el inicio de este.
    * Remover la información previa a la etiqueta
    * Remover la información antes del inicio del texto y despues de la etiqueta
* Eliminar la información extra por hoja.
    * Eliminar la información extra de la primera pagina (referencias y pie de pagina)
    * Eliminar la información extra de la segunda pagina (referencias, preguntas y pie de pagina)
* Eliminar las referencias a imagenes aún existentes.

## Resultado esperado

Finalmente, se espera obtener un arreglo de elementos donde cada represente a un texto completo y que este listo para ser pre-procesado.

In [0]:
def findTextBeginning(text):
    firstPage = text[0]
    currentPosition = -1
    for index, paragraph in enumerate(firstPage.split('\n')):
        print(index, paragraph, '\n\n')
    
    for TAG in ALLOWED_TAGS:
        currentPosition = firstPage.find(TAG) if currentPosition == -1 else currentPosition
    
    print(firstPage[0:currentPosition])

In [0]:
findTextBeginning(cleanTextArr[0])

0                                                                                  1.er grado de secundaria 


1              Ficha: Reconocemos la estructura del texto 


2                                 argumentativo 


3                   APRENDEMOS 


4                   Lee el siguiente texto considerando las orientaciones que brinda tu docente. 


5                                                    El acto solidario de la donación de órganos1 


6                                                                                                        Fuente de imagen: <https://goo.gl/exLu3M> 


7                               Si bien los trasplantes se han convertido en una práctica habitual, aún persisten fuertes 


8                               temores en la población para donar órganos, lograr su superación es la clave para aumentar el 


9                               número de los donadores solidarios que hacen falta para salvar miles de vidas. 


10                      